In [1]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [2]:
data = pd.read_csv('shedulers.csv', sep=';', encoding='cp1251')
print(data.info())
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ФИО                        8 non-null      object
 1   Расписание                 8 non-null      object
 2   Дата начала расписания     8 non-null      object
 3   Дата окончания расписания  8 non-null      object
dtypes: object(4)
memory usage: 384.0+ bytes
None


,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


## Задание 1

Создать таблицу (T_CONTRACTOR_SHERULER) под расписание и заполнить его с файла schedulers.csv. (Использовать любую библиотеку для заливки в СУБД (как вариант MS SQL))  
Таблицу следует нормализовать.

- ID_NAME - идентификтор поставщика
- NAME - название поставщика
- SHEDULE - расписание
- DATE_BEGIN - дата начала действия расписания
- DATE_END - дата окончания действия расписания

Пример записи без нормализации:
Поставщик 1 ДВС 01.01.2019 04.01.2019
Поставщик 2 НВС 05.01.2019 31.12.2019

Вводные:
- Связку полей FIO, DATE_BEGIN считать уникальной.
- DATE_BEGIN не может привышать DATE_END.
- Можете продемонстрировать работу с ключами/ограничениями

Приводим данные к правильному виду

In [3]:
data.columns = ['NAME','SHEDULE', 'DATE_BEGIN', 'DATE_END']
data['DATE_BEGIN'] = pd.to_datetime(data['DATE_BEGIN'], dayfirst=True, errors='coerce').fillna(dt.datetime(2099,12,31))
data['DATE_END'] = pd.to_datetime(data['DATE_END'], dayfirst=True, errors='coerce').fillna(dt.datetime(2099,12,31))
data

,NAME,SHEDULE,DATE_BEGIN,DATE_END
0,Поставщик 1,дддвсвнн,2019-01-01,2019-01-10
1,Поставщик 1,ннвннв,2019-01-11,2019-01-15
2,Поставщик 1,св,2019-01-16,2019-01-20
3,Поставщик 2,свсвсв,2019-01-01,2019-01-07
4,Поставщик 2,днвсв,2019-01-08,2019-01-14
5,Поставщик 2,ннддвсв,2019-01-15,2099-12-31
6,Поставщик 3,нвнвнв,2019-01-01,2019-02-01
7,Поставщик 3,двдвдвдв,2019-02-02,2099-12-31


Создаем базу данных и таблицу

In [4]:
con = pymysql.connect(host='localhost', user='root', password='')
try:
    cursor = con.cursor()
    cursor.execute("CREATE DATABASE sheduler")
except Exception as e:
    print("Exeception occured:{}".format(e))
finally:
    con.close()

In [16]:
con = pymysql.connect(host='localhost', user='root', password='', database='sheduler')
try:
    cursor = con.cursor()
    cursor.execute("drop table if exists T_CONTRACTOR_SHERULER")
    query = "CREATE TABLE T_CONTRACTOR_SHERULER(ID_NAME int PRIMARY KEY AUTO_INCREMENT, \
             NAME varchar(32) NOT NULL, SHEDULE varchar(32) NOT NULL, DATE_BEGIN date NOT NULL, DATE_END date NOT NULL, \
             CONSTRAINT name_date UNIQUE (NAME, DATE_BEGIN), \
             CONSTRAINT dates CHECK (DATE_BEGIN<=DATE_END))"
    cursor.execute(query)
    cursor.execute("show tables")
    tables = cursor.fetchall()
    for table in tables:
        print(table)
except Exception as e:
    print("Exeception occured:{}".format(e))
finally:
    con.close()

('t_contractor_sheruler',)
('t_contractor_work_day',)


Самый простой вариант записать данные из pandas DataFrame в таблицу - методом pd.to_sql

In [17]:
engine = create_engine("mysql+pymysql://root:@localhost/sheduler", echo=False)
data.to_sql('T_CONTRACTOR_SHERULER', con=engine, if_exists='append', index=False)

C:\Users\albri\.conda\envs\python38\lib\site-packages\pandas\io\sql.py:1422: UserWarning: The provided table name 'T_CONTRACTOR_SHERULER' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


Другой вариант - SQL-запросами через pymysql

In [7]:
con = pymysql.connect(host='localhost', user='root', password='', database='sheduler')
try:
    cursor = con.cursor()
    cursor.execute("truncate table T_CONTRACTOR_SHERULER")
    
    for ind, row in data.iterrows():
        date1 = dt.date.strftime(row['DATE_BEGIN'], '%Y-%m-%d')
        date2 = dt.date.strftime(row['DATE_END'], '%Y-%m-%d')
        query = 'INSERT INTO T_CONTRACTOR_SHERULER VALUES (DEFAULT, %s, %s, %s, %s)'
        cursor.execute(query, (row['NAME'], row['SHEDULE'], date1, date2))
    con.commit()
    
except Exception as e:
    print("Exeception occured:{}".format(e))
finally:
    con.close()

## Задание 2
Создать таблицу (T_CONTRACTOR_WORK_DAY) выходов на работу сотрудников.
Таблица должна иметь следующий вид:
- ID - идентификатор записи
- NAME - название поставщика
- DATE_BEGIN - Начало рабочего дня (datetime)
- DATE_END - Конец рабочего дня (datetime)

In [8]:
con = pymysql.connect(host='localhost', user='root', password='', database='sheduler')
try:
    cursor = con.cursor()
    cursor.execute("drop table if exists T_CONTRACTOR_WORK_DAY")
    query = "CREATE TABLE T_CONTRACTOR_WORK_DAY(ID int PRIMARY KEY AUTO_INCREMENT, NAME varchar(32) NOT NULL, \
             DATE_BEGIN datetime NOT NULL, DATE_END datetime NOT NULL, \
             CONSTRAINT name_date UNIQUE (NAME, DATE_BEGIN))"
    cursor.execute(query)
except Exception as e:
    print("Exeception occured:{}".format(e))
finally:
    con.close()

## Задание 3
Создать процедуру расчета рабочих дней.

Входящие параметры:
- Дата начала периода расчета
- Дата окончания периода расчета.

Ожидаемый результать выполнения хранимой процедуры - заполнение таблицы T_CONTRACTOR_WORK_DAY рабочими днями согласно расписания работы поставщиков из таблицы T_CONTRACTOR_SHERULER. Выходные дни (В) не должны попадать в таблицу T_CONTRACTOR_WORK_DAY

Пример выполнения для Поставщика 1 (Из примера записи таблицы T_CONTRACTOR_SHERULER) с параметрами '01.01.2019' - '08.01.2019'. Таблица T_CONTRACTOR_WORK_DAY заполнится следующими данными:
- 1 Поставщик 1 01.01.2019 08:00 01.01.2019 20:00
- 2 Поставщик 1 03.01.2019 08:00 04.01.2019 08:00
- 3 Поставщик 1 04.01.2019 08:00 04.01.2019 08:00
- 4 Поставщик 1 05.01.2019 20:00 06.01.2019 08:00
- 5 Поставщик 1 07.01.2019 08:00 08.01.2019 08:00
- 6 Поставщик 1 08.01.2019 20:00 09.01.2019 08:00

Пояснение: для записей с 01.01.2019 по 04.01.2019 берется расписание ДВС

- 1 - Д - дневная смена далее следует выходной В - запись о выходном дне не попадает в таблицу
- 2 - С - суточная смена
- 3 - расписание закончилось, поэтому оно циклично начинается с начала (Д - дневная смена)
- 4 - С 05.01.2019 начинает действовать новое расписание - НВС берется Н - ночная смена далее следует выходной В - запись о выходном дне не попадает в таблицу
- 5 - С - суточная смена
- 6 - Снова Н - ночная смена

```SQL
delimiter //
CREATE PROCEDURE worktable(sdate1 CHAR(10), sdate2 CHAR(10))
BEGIN
  DECLARE date1 DATE DEFAULT STR_TO_DATE(sdate1, "%d.%m.%Y");
  DECLARE date2 DATE DEFAULT STR_TO_DATE(sdate2, "%d.%m.%Y");
  DECLARE cur_date, row_date1, row_date2 DATE;
  DECLARE shift INT;
  DECLARE row_name, row_shed char(20);
  DECLARE done INT DEFAULT FALSE;
  
  -- сразу выберем только те строки, которые попадают в нужные даты
  DECLARE cur CURSOR FOR SELECT name,shedule,date_begin,date_end 
                         FROM T_CONTRACTOR_SHERULER 
                         WHERE NOT (date1>date_end OR date2<date_begin);
                         
  -- открываем курсор
  DECLARE CONTINUE HANDLER FOR NOT FOUND SET done=TRUE;
  OPEN cur;
  
  -- цикл по строкам
  row_loop: WHILE NOT done DO
     FETCH cur INTO row_name, row_shed, row_date1, row_date2;
     IF NOT done THEN
        -- определяем первый нужный день внутри строки
     	SET cur_date = date1;
        IF row_date1 > date1 THEN
          SET cur_date = row_date1;
        end IF;
        
     	-- цикл по датам внутри строки
        date_loop: WHILE cur_date <= row_date2 AND cur_date <= date2 DO
          -- определяем тип расписания для текущей даты
          SET shift = datediff(cur_date, row_date1) % CHAR_LENGTH(row_shed);
          CASE substring(row_shed, shift+1, 1)
            WHEN 'д' THEN INSERT INTO t_contractor_work_day VALUES (DEFAULT, row_name, date_add(cur_date, INTERVAL 8 hour), date_add(cur_date, interval 20 hour));
            WHEN 'н' THEN INSERT INTO t_contractor_work_day VALUES (DEFAULT, row_name, date_add(cur_date, INTERVAL 20 hour), date_add(cur_date, interval 32 hour));
            WHEN 'с' THEN INSERT INTO t_contractor_work_day VALUES (DEFAULT, row_name, date_add(cur_date, INTERVAL 8 hour), date_add(cur_date, interval 32 hour));
            ELSE BEGIN END; 
          END CASE;
          SET cur_date = date_add(cur_date, INTERVAL 1 day);
        END WHILE date_loop;
        
     END IF;
  END WHILE row_loop;
  CLOSE cur;
END //

delimiter ;
```

## Задание 4
С помощью SQL запросов:
- Сделать выборку содержащую сколько рабочих дней было у каждого поставщика
- Сделать выборку поставщиков, у которых было больше 10 рабочих дней за январь 2019 года
- Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

Сколько рабочих дней у каждого поставщика

```sql
SELECT name, count(*) as 'work days'
FROM t_contractor_work_day
group by name
```

Выборка поставщиков, у которых было больше 10 рабочих дней за январь 2019 года

```sql
SELECT name, count(*) as 'work days'
FROM t_contractor_work_day
where date_begin between '2019-01-01' and '2019-01-31 23:59:59'
group by name
having count(*) > 10
```

Второй вариант, если нужно вывести только список постащиков без указания кол-ва рабочих дней

```sql
SELECT name
FROM (
    SELECT name, count(*) AS 'work days'
    FROM t_contractor_work_day
    WHERE date_begin BETWEEN '2019-01-01' AND '2019-01-31 23:59:59'
    GROUP BY name
    HAVING count(*) > 10) AS t
```

Выборка поставщиков, кто работал 14, 15 и 16 января 2019 года (предполагаю, что имеется в виду любой из этих дней)

```sql
SELECT *
FROM t_contractor_work_day
WHERE date_begin BETWEEN '2019-01-14' AND '2019-01-16 23:59:59'
   or date_end BETWEEN '2019-01-14' AND '2019-01-16 23:59:59'
```

Запрос выше учитывает в том числе вечерние и суточные смены, которые начались 13 числа и заканчиваются 16. Если нужно взять только те смены, которые _начинаются_ 14, 15 или 16 января, достаточно удалить проверку условия date_end.  
Вариант, если нужно вывести только список поставщиков без указания смен:

```sql
SELECT distinct name
FROM t_contractor_work_day
WHERE date_begin BETWEEN '2019-01-14' AND '2019-01-16 23:59:59'
   or date_end BETWEEN '2019-01-14' AND '2019-01-16 23:59:59'
```

## Задание 5
Выполните задачи 3, 4 с помощью python.

In [9]:
def calc_work_days(date1, date2):

    date1 = dt.datetime.strptime(date1,'%d.%m.%Y')
    date2 = dt.datetime.strptime(date2,'%d.%m.%Y')
    subdata = data[~((date1>data['DATE_END']) | (date2<data['DATE_BEGIN']))]
    work_days = []

    for ind, row in subdata.iterrows():

        date_sub1 = date1 if date1>row['DATE_BEGIN'] else row['DATE_BEGIN']
        date_sub2 = date2 if date2<row['DATE_END'] else row['DATE_END']

        for date in pd.date_range(date_sub1, date_sub2):

            shift = (date - row['DATE_BEGIN']).days
            shift %= len(row['SHEDULE'])
            shed = row['SHEDULE'][shift]

            if shed=='д':
                work_days.append([row['NAME'], date+dt.timedelta(hours=8), date+dt.timedelta(hours=20)])
            if shed=='н':
                work_days.append([row['NAME'], date+dt.timedelta(hours=20), date+dt.timedelta(hours=32)])
            if shed=='с':
                work_days.append([row['NAME'], date+dt.timedelta(hours=8), date+dt.timedelta(hours=32)])

    return pd.DataFrame(work_days, columns=['NAME','DATE_BEGIN','DATE_END'])

In [10]:
work_days = calc_work_days('01.01.2019', '28.02.2019')
work_days

,NAME,DATE_BEGIN,DATE_END
0,Поставщик 1,2019-01-01 08:00:00,2019-01-01 20:00:00
1,Поставщик 1,2019-01-02 08:00:00,2019-01-02 20:00:00
2,Поставщик 1,2019-01-03 08:00:00,2019-01-03 20:00:00
3,Поставщик 1,2019-01-05 08:00:00,2019-01-06 08:00:00
4,Поставщик 1,2019-01-07 20:00:00,2019-01-08 08:00:00
...,...,...,...
82,Поставщик 3,2019-02-20 08:00:00,2019-02-20 20:00:00
83,Поставщик 3,2019-02-22 08:00:00,2019-02-22 20:00:00
84,Поставщик 3,2019-02-24 08:00:00,2019-02-24 20:00:00
85,Поставщик 3,2019-02-26 08:00:00,2019-02-26 20:00:00


Количество рабчих дней у каждого поставщика

In [11]:
work_days['NAME'].value_counts()

Поставщик 2    42
Поставщик 3    30
Поставщик 1    15
Name: NAME, dtype: int64

Выборка поставщиков, у которых было больше 10 рабочих дней за январь 2019 года

In [12]:
work_days[(work_days['DATE_BEGIN'] >= dt.datetime(2019,1,1)) 
          & (work_days['DATE_BEGIN'] < dt.datetime(2019,2,1))]['NAME'].value_counts()

Поставщик 2    22
Поставщик 3    16
Поставщик 1    15
Name: NAME, dtype: int64

In [13]:
work_days[(work_days['DATE_BEGIN'] >= dt.datetime(2019,1,1)) 
          & (work_days['DATE_BEGIN'] < dt.datetime(2019,2,1))]['NAME'].value_counts().index

Index(['Поставщик 2', 'Поставщик 3', 'Поставщик 1'], dtype='object')

Выборка поставщиков, кто работал 14, 15 и 16 января 2019 года

In [14]:
start_date = dt.datetime(2019,1,14)
end_date = dt.datetime(2019,1,16,23,59,59)
work_days[(work_days['DATE_BEGIN'].between(start_date, end_date)) | 
          (work_days['DATE_END'].between(start_date, end_date))]

,NAME,DATE_BEGIN,DATE_END
10,Поставщик 1,2019-01-14 20:00:00,2019-01-15 08:00:00
11,Поставщик 1,2019-01-15 20:00:00,2019-01-16 08:00:00
12,Поставщик 1,2019-01-16 08:00:00,2019-01-17 08:00:00
23,Поставщик 2,2019-01-14 20:00:00,2019-01-15 08:00:00
24,Поставщик 2,2019-01-15 20:00:00,2019-01-16 08:00:00
25,Поставщик 2,2019-01-16 20:00:00,2019-01-17 08:00:00
63,Поставщик 3,2019-01-13 20:00:00,2019-01-14 08:00:00
64,Поставщик 3,2019-01-15 20:00:00,2019-01-16 08:00:00


In [15]:
start_date = dt.datetime(2019,1,14)
end_date = dt.datetime(2019,1,16,23,59,59)
work_days[(work_days['DATE_BEGIN'].between(start_date, end_date)) | 
          (work_days['DATE_END'].between(start_date, end_date))]['NAME'].unique()

array(['Поставщик 1', 'Поставщик 2', 'Поставщик 3'], dtype=object)